<a href="https://colab.research.google.com/github/budennovsk/Pandas/blob/master/mesa_ABM_ipywidgets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mesa ipywidgets


  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.9 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [27]:
import mesa
import pandas as pd
import random
import matplotlib.pyplot as plt
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from ipywidgets import interact, IntSlider

# Данные
data = pd.DataFrame({
    "sku": ["sku1", "sku2", "sku3", "sku4", "sku5", "sku6", "sku7", "sku8", "sku9", "sku10"],
    "share_vol": [0.1, 0.2, 0.15, 0.05, 0.25, 0.25, 0.07, 0.04, 0.12, 0.08],
    "price": [5, 4, 6, 8, 3, 4, 2, 10, 9, 1],
    "sales_vol": [100, 150, 120, 80, 200, 100, 70, 40, 90, 60],
})

# Агенты-потребители
class Consumer(Agent):
    """Агент-потребитель."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def step(self):
        # Агенты выбирают SKU для покупки на основе веса
        choice = random.choices(self.model.skus, weights=self.model.weights, k=1)[0]
        self.model.sales_history[choice] += 1  # Обновляем историю продаж

        # Передвижение агента по сетке
        possible_moves = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = random.choice(possible_moves)
        self.model.grid.move_agent(self, new_position)

# Модель рынка
class Market(Model):
    """Модель рынка с агентами."""

    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, True)  # Создаем сетку
        self.skus = data["sku"].tolist()
        self.weights = data["share_vol"].tolist()

        # История продаж для каждого SKU
        self.sales_history = {sku: 0 for sku in self.skus}
        self.sales_over_time = {sku: [] for sku in self.skus}

        # Добавляем агентов на сетку
        for i in range(self.num_agents):
            a = Consumer(i, self)
            self.schedule.add(a)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

    def step(self):
        # Один шаг модели - агенты совершают покупки и перемещаются
        self.schedule.step()
        # Обновляем историю продаж для каждого SKU
        for sku in self.skus:
            self.sales_over_time[sku].append(self.sales_history[sku])

# Инициализация модели: 100 агентов на сетке 10x10
model = Market(100, 10, 10)

# Симуляция на 30 дней
days = 30
for day in range(days):
    model.step()

# Визуализация с использованием ipywidgets и matplotlib
def plot_sales(selected_day):
    fig, ax = plt.subplots(figsize=(10, 6))
    for sku in model.skus:
        ax.plot(range(selected_day + 1), model.sales_over_time[sku][:selected_day + 1], label=sku)

    ax.set_xlabel("Days")
    ax.set_ylabel("Sales Volume")
    ax.set_title(f"Sales Volume Over Time (up to day {selected_day})")
    ax.legend()
    plt.show()

# Ползунок для выбора дня
interact(plot_sales, selected_day=IntSlider(min=0, max=days - 1, step=1, value=0))


interactive(children=(IntSlider(value=0, description='selected_day', max=29), Output()), _dom_classes=('widget…

<function __main__.plot_sales(selected_day)>

In [10]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
import random

# Агент-потребитель
class ConsumerAgent(Agent):
    def __init__(self, unique_id, model, price_sensitivity, brand_preference, product_preference):
        super().__init__(unique_id, model)
        self.price_sensitivity = price_sensitivity  # чувствительность к цене
        self.brand_preference = brand_preference  # предпочтение по бренду (0-1, где 1 - премиум)
        self.product_preference = product_preference  # предпочтение по свойствам продукта (например, жирность)

    def step(self):
        # На каждом шаге агент выбирает продукт для покупки
        best_product = None
        best_score = -float('inf')

        # Проходим по каждому доступному продукту (SKU)
        for product in self.model.products:
            # Расчет оценки продукта на основе цены, бренда и свойств продукта
            price_score = -self.price_sensitivity * product['price']
            brand_score = (self.brand_preference - product['brand_level']) ** 2
            property_score = -abs(self.product_preference - product['properties'])

            total_score = price_score + property_score - brand_score

            if total_score > best_score:
                best_score = total_score
                best_product = product

        # Если агент выбрал продукт, увеличиваем его долю на рынке
        if best_product:
            best_product['market_share'] += 1

# Модель рынка
class MarketModel(Model):
    def __init__(self, N, width, height, product_data):
        super().__init__()
        self.num_agents = N  # Количество потребителей
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

        # Список продуктов (SKU) с их свойствами
        self.products = product_data

        # Создаем агентов-потребителей
        for i in range(self.num_agents):
            price_sensitivity = random.uniform(0.5, 1.5)
            brand_preference = random.uniform(0, 1)  # 0 = бюджетные, 1 = премиум
            product_preference = random.uniform(0, 100)  # Предпочтение по свойствам продукта (например, жирность)
            consumer = ConsumerAgent(i, self, price_sensitivity, brand_preference, product_preference)
            self.schedule.add(consumer)

    def step(self):
        # Выполняем один шаг моделирования
        for product in self.products:
            product['market_share'] = 0  # Сбрасываем долю рынка для каждого SKU
        self.schedule.step()

        # Меняем цены продуктов для имитации рыночных изменений
        for product in self.products:
            product['price'] *= random.uniform(0.95, 1.05)  # Колебание цен

        # Динамическое добавление или удаление продуктов
        if random.random() < 0.1:  # С вероятностью 10% добавляем новый продукт
            new_product = {
                'name': f'NewProduct{len(self.products)+1}',
                'price': random.uniform(50, 100),
                'brand_level': random.uniform(0, 1),  # Премиум или бюджет
                'properties': random.uniform(0, 100),  # Свойства продукта (например, вкус, состав)
                'market_share': 0
            }
            self.products.append(new_product)

        if random.random() < 0.05:  # С вероятностью 5% удаляем продукт
            if self.products:
                self.products.pop(random.randint(0, len(self.products) - 1))

# Запуск модели с набором продуктов
initial_products = [
    {'name': 'Сыр1', 'price': 50, 'brand_level': 0.3, 'properties': 70, 'market_share': 0},
    {'name': 'Сыр2', 'price': 80, 'brand_level': 0.8, 'properties': 50, 'market_share': 0},
    {'name': 'Сыр3', 'price': 60, 'brand_level': 0.5, 'properties': 90, 'market_share': 0}
]

market = MarketModel(10, 10, 10, initial_products)

# Моделируем 10 временных шагов
for i in range(10):
    print(f"Step {i+1}")
    market.step()

    # Печать долей рынка каждого SKU
    for product in market.products:
        print(f"{product['name']} - Доля рынка: {product['market_share']}")

Step 1
Сыр1 - Доля рынка: 8
Сыр2 - Доля рынка: 0
Сыр3 - Доля рынка: 2
NewProduct4 - Доля рынка: 0
Step 2
Сыр1 - Доля рынка: 4
Сыр2 - Доля рынка: 0
Сыр3 - Доля рынка: 2
NewProduct4 - Доля рынка: 4
Step 3
Сыр1 - Доля рынка: 4
Сыр2 - Доля рынка: 0
Сыр3 - Доля рынка: 2
NewProduct4 - Доля рынка: 4
Step 4
Сыр1 - Доля рынка: 4
Сыр2 - Доля рынка: 0
Сыр3 - Доля рынка: 2
NewProduct4 - Доля рынка: 4
Step 5
Сыр1 - Доля рынка: 4
Сыр2 - Доля рынка: 0
Сыр3 - Доля рынка: 2
NewProduct4 - Доля рынка: 4
NewProduct5 - Доля рынка: 0
Step 6
Сыр1 - Доля рынка: 4
Сыр2 - Доля рынка: 0
Сыр3 - Доля рынка: 2
NewProduct4 - Доля рынка: 4
NewProduct5 - Доля рынка: 0
Step 7
Сыр1 - Доля рынка: 4
Сыр2 - Доля рынка: 0
Сыр3 - Доля рынка: 2
NewProduct4 - Доля рынка: 4
NewProduct5 - Доля рынка: 0
Step 8
Сыр1 - Доля рынка: 4
Сыр2 - Доля рынка: 0
Сыр3 - Доля рынка: 2
NewProduct4 - Доля рынка: 4
NewProduct5 - Доля рынка: 0
Step 9
Сыр1 - Доля рынка: 4
Сыр2 - Доля рынка: 0
Сыр3 - Доля рынка: 2
NewProduct4 - Доля рынка: 4
NewPro

In [39]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
import random
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# Агент-потребитель
class ConsumerAgent(Agent):
    def __init__(self, unique_id, model, price_sensitivity, brand_preference, product_preference):
        super().__init__(unique_id, model)
        self.price_sensitivity = price_sensitivity  # чувствительность к цене
        self.brand_preference = brand_preference  # предпочтение по бренду (0-1, где 1 - премиум)
        self.product_preference = product_preference  # предпочтение по свойствам продукта (например, жирность)

    def step(self):
        # На каждом шаге агент выбирает продукт для покупки
        best_product = None
        best_score = -float('inf')

        # Проходим по каждому доступному продукту (SKU)
        for product in self.model.products:
            # Расчет оценки продукта на основе цены, бренда и свойств продукта
            price_score = -self.price_sensitivity * product['price']
            brand_score = (self.brand_preference - product['brand_level']) ** 2
            property_score = -abs(self.product_preference - product['properties'])

            total_score = price_score + property_score - brand_score

            if total_score > best_score:
                best_score = total_score
                best_product = product

        # Если агент выбрал продукт, увеличиваем его долю на рынке
        if best_product:
            best_product['market_share'] += 1

# Модель рынка
class MarketModel(Model):
    def __init__(self, N, width, height, product_data):
        super().__init__()
        self.num_agents = N  # Количество потребителей
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

        # Список продуктов (SKU) с их свойствами
        self.products = product_data

        # Создаем агентов-потребителей
        for i in range(self.num_agents):
            price_sensitivity = random.uniform(0.5, 1.5)
            brand_preference = random.uniform(0, 1)  # 0 = бюджетные, 1 = премиум
            product_preference = random.uniform(0, 100)  # Предпочтение по свойствам продукта (например, жирность)
            consumer = ConsumerAgent(i, self, price_sensitivity, brand_preference, product_preference)
            self.schedule.add(consumer)

        # Сохраняем историю долей рынка
        self.market_share_history = {product['name']: [] for product in self.products}

    def step(self):
        # Выполняем один шаг моделирования
        for product in self.products:
            product['market_share'] = 0  # Сбрасываем долю рынка для каждого SKU
        self.schedule.step()

        # Меняем цены продуктов для имитации рыночных изменений
        for product in self.products:
            product['price'] *= random.uniform(0.95, 1.05)  # Колебание цен

        # Динамическое добавление или удаление продуктов
        if random.random() < 0.1:  # С вероятностью 10% добавляем новый продукт
            new_product = {
                'name': f'NewProduct{len(self.products)+1}',
                'price': random.uniform(50, 100),
                'brand_level': random.uniform(0, 1),  # Премиум или бюджет
                'properties': random.uniform(0, 100),  # Свойства продукта (например, вкус, состав)
                'market_share': 0
            }
            self.products.append(new_product)
            self.market_share_history[new_product['name']] = []

        if random.random() < 0.05:  # С вероятностью 5% удаляем продукт
            if self.products:
                removed_product = self.products.pop(random.randint(0, len(self.products) - 1))
                del self.market_share_history[removed_product['name']]

        # Сохраняем доли рынка на текущем шаге
        for product in self.products:
            self.market_share_history[product['name']].append(product['market_share'])

# Запуск модели с набором продуктов
initial_products = [
    {'name': 'Сыр1', 'price': 50, 'brand_level': 0.3, 'properties': 70, 'market_share': 0},
    {'name': 'Сыр2', 'price': 80, 'brand_level': 0.8, 'properties': 50, 'market_share': 0},
    {'name': 'Сыр3', 'price': 60, 'brand_level': 0.5, 'properties': 90, 'market_share': 0}
]

market = MarketModel(10, 10, 10, initial_products)

# Моделируем 30 временных шагов
for i in range(30):
    market.step()

# Функция для построения графика
def plot_market_share(selected_day):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Находим максимальную длину списка долей рынка
    max_length = max(len(market_share) for market_share in market.market_share_history.values())

    for product_name, market_share in market.market_share_history.items():
        # Обрезка или дополнение списка долей рынка до максимальной длины
        if len(market_share) < max_length:
            market_share += [0] * (max_length - len(market_share))
        ax.plot(range(selected_day + 1), market_share[:selected_day + 1], label=product_name)

    ax.set_xlabel("Days")
    ax.set_ylabel("Market Share")
    ax.set_title(f"Market Share Over Time (up to day {selected_day})")
    ax.legend()
    plt.show()

# Ползунок для выбора дня
interact(plot_market_share, selected_day=IntSlider(min=0, max=29, step=1, value=0))

interactive(children=(IntSlider(value=0, description='selected_day', max=29), Output()), _dom_classes=('widget…

<function __main__.plot_market_share(selected_day)>

In [42]:
import pandas as pd

# Исторические данные по продажам (цены, свойства продукта, бренд, продажи)
data = pd.DataFrame({
    'sku': ['Сыр1', 'Сыр2', 'Сыр3'],
    'price': [50, 80, 60],
    'brand_level': [0.3, 0.8, 0.5],
    'properties': [70, 50, 90],
    'sales': [100, 150, 120]  # Исторические продажи
})
'''Функция для расчета рыночной доли на основе текущих параметров:
python
Копировать код'''
def market_share(price_sensitivity, brand_preference, product_preference, product):
    price_score = -price_sensitivity * product['price']
    brand_score = (brand_preference - product['brand_level']) ** 2
    property_score = -abs(product_preference - product['properties'])
    return price_score + brand_score + property_score
'''Оптимизация параметров на основе исторических данных:
python
Копировать код'''
from scipy.optimize import minimize

# Целевая функция для минимизации (разница между симуляцией и историческими продажами)
def objective_function(params):
    price_sensitivity, brand_preference, product_preference = params
    simulated_sales = []

    for i, product in data.iterrows():
        # Оценка рыночной доли на основе текущих параметров
        score = market_share(price_sensitivity, brand_preference, product_preference, product)
        simulated_sales.append(score)

    # Приведение продаж к масштабируемым значениям и расчет ошибки
    simulated_sales = [s / sum(simulated_sales) * sum(data['sales']) for s in simulated_sales]
    error = sum((simulated_sales[i] - data['sales'][i]) ** 2 for i in range(len(simulated_sales)))
    return error

# Начальные предположения для параметров
initial_params = [1.0, 0.5, 50]

# Оптимизация параметров модели
result = minimize(objective_function, initial_params, method='Nelder-Mead')
best_params = result.x
print(f"Оптимизированные параметры: {best_params}")

Оптимизированные параметры: [ 14.02999999  -3.10440459 -34.58117288]
